In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import nengo

In [3]:
class STDE3_wInhibition(object):
    def __init__(self, 
                 w_fac = 1, 
                 w_trig = 0.1, 
                 tau_fac1=0.0005,
                 tau_fac2=0.0005,
                 tau_fac3=0.0005,
                 tau_trig=0.005,
                 soma_type=nengo.LIF(),
                 ):
        assert w_fac.shape == w_trig.shape    
        N_neurons, N_input = w_fac.shape
        self.model = nengo.Network()
        with self.model:
            
            self.input = nengo.Node(None, size_in=N_input)
            
            self.dendrite1 = nengo.Node(lambda t, x: x[:N_neurons]*x[N_neurons:], size_in=N_neurons*2)
            nengo.Connection(self.input, self.dendrite1[:N_neurons], transform=w_fac, synapse=tau_fac1)
            nengo.Connection(self.input, self.dendrite1[N_neurons:], transform=w_trig, synapse=None)

            
            self.dendrite2 = nengo.Node(lambda t, x: x[:N_neurons]*x[N_neurons:], size_in=N_neurons*2)
            nengo.Connection(self.input, self.dendrite2[:N_neurons], transform=w_fac, synapse=tau_fac2)
            nengo.Connection(self.input, elf.dendrite2[N_neurons:], transform=w_trig, synapse=None)

            
            self.dendrite3 = nengo.Node(lambda t, x: x[:N_neurons]*x[N_neurons:], size_in=N_neurons*2)
            nengo.Connection(self.input, self.dendrite3[:N_neurons], transform=w_fac, synapse=tau_fac3)
            nengo.Connection(self.input, self.dendrite3[N_neurons:], transform=w_trig, synapse=None)
            
            self.neurons = nengo.Ensemble(n_neurons=N_neurons*3, dimensions=1,
                                          neuron_type=soma_type, 
                                          gain=np.ones(N_neurons*3), bias=np.zeros(N_neurons*3))
            nengo.Connection(self.dendrite1, self.neurons.neurons[0: N_neurons*3: 3], synapse=tau_trig, transform = 1/10.0)
            nengo.Connection(self.dendrite2, self.neurons.neurons[1: N_neurons*3: 3], synapse=tau_trig, transform = 1/10.0)
            nengo.Connection(self.dendrite3, self.neurons.neurons[2: N_neurons*3: 3], synapse=tau_trig, transform = 1/10.0)
            
#             self.inhib = nengo.Ensemble(n_neurons=1, dimensions=1, neuron_type=soma_type, 
#                                           gain=np.ones(1), bias=np.zeros(1))
#             nengo.Connection(self.neurons.neurons, self.inhib.neurons, transform = np.ones((1,N_neurons*3)), synapse=0.005)
#             nengo.Connection(self.inhib.neurons, self.neurons.neurons, transform = (-0.05)*np.ones((N_neurons*3, 1)), synapse=0.005)

            a = np.ones((N_neurons*3,N_neurons*3))
            for i in range(N_neurons*3):
                a[i][i] = 0
            nengo.Connection(self.neurons.neurons, self.neurons.neurons, transform = (-0.08)*a)

            
            self.output = nengo.Node(None, size_in=N_neurons*3)
            nengo.Connection(self.neurons.neurons, self.output, synapse=None)
            
    def run(self, input, input_dt=0.001, output_dt=0.001, Simulator=nengo.Simulator, progress_bar=True):
        model = nengo.Network()
        model.networks.append(self.model)
        with model:
            stim = nengo.Node(nengo.processes.PresentInput(input, presentation_time=input_dt))
            nengo.Connection(stim, self.input, synapse=None)
            p = nengo.Probe(self.output, sample_every=output_dt)
        sim = Simulator(model, progress_bar=progress_bar, dt=1/100e3)
        with sim:
            sim.run(len(input)*input_dt)
        return sim.data[p]

In [7]:
w_fac = 1
w_trig = 1/10
tau_fac = 0.0001
tau_trig = 0.005
soma_type = nengo.LIF(tau_ref=0.001)

number_stdes = 20

w_fac = np.diag(np.ones(number_stdes))* w_trig
w_trig = np.diag(np.ones(number_stdes))[::-1]* w_trig

N_neurons, N_input = w_fac.shape
print("Number neurons = " + str(N_neurons) + ".  Number inputs = " + str(N_input))


m = STDE3_wInhibition(w_fac=w_fac, w_trig=w_trig)


model = m.model

import nengo_gui.jupyter
nengo_gui.jupyter.InlineGUI(model)

Number neurons = 20.  Number inputs = 20


In [9]:
w_trig

array([[0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0.1],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0.1, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0.1, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0.1, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0.1, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0.1, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0.1, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.1,
        0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0